In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#function that will filter out error triangles
def generate_neighborhood(triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def set_search_first(starting_node, neighborhood):
    """
    Modified Depth-First-Search utilizing sets to reduce duplicate checks:

    Neighborhood must be a dict with the keys being the vertex indices!
    """    
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return list(visited_nodes)
def get_connected_portions(neighborhood):
    neighborhood_copy = neighborhood.copy()
    portions = []
    while len(neighborhood_copy) > 0:
        starting_node = next(iter(neighborhood_copy))
        portion = set_search_first(starting_node, neighborhood_copy)
        for node in portion:
            neighborhood_copy.pop(node)
        portions.append(portion)
    return portions

def get_largest_portion_index(portions):
    portion_lengths = [len(portion) for portion in portions]
    return portion_lengths.index(max(portion_lengths))

def get_largest_portion(portions):
    return portions[get_largest_portion_index(portions)]

def remove_non_soma(mesh,key,mesh_labels):    
    mesh_copy = mesh.copy()
    
#     #get the labels for the mesh
#     #find out if in Orphan Table or Regular Neuron Table
#     if len(ta3p100.CoarseLabelFinal() & key) > 0:
#         mesh_labels = (ta3p100.CoarseLabelFinal & key).fetch1()
#     elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
#         mesh_labels = (ta3p100.CoarseLabelOrphan & key).fetch1()
#     else:
#         raise Exception('neuron' + str(key["segment_id"]) + 
#                         'not present in any labels!')

    
    #look for errors
    not_errors = [i for i,k in enumerate(mesh_labels["triangles"]) if k == 5]
    original_triangles = mesh["triangles"]
    """
    print(type(not_errors))
    print(len(not_errors))
    print("not_errors = "+ str(not_errors[:100]))
    print(type(original_triangles))
    print(len(original_triangles))
    #print(original_triangles)
    print("not_errors = " + str(original_triangles[not_errors]))
    """
    
    mesh_copy['triangles'] = np.array(original_triangles[not_errors])
    
    return mesh_copy


def remove_isolated_vertices(mesh):
    mesh_copy = mesh.copy()

    neighborhood = generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    isolated_nodes = [portion.pop() for portion in get_connected_portions(neighborhood) if len(portion) == 1]

    vertices = mesh_copy['vertices']
    triangles = mesh_copy['triangles']
    vertex_list = list(vertices)

    if len(isolated_nodes) > 0:
        num_isolated_nodes_passed = 0
        isolated_nodes_set = set(isolated_nodes)
        count_to_decrement = np.zeros(len(vertices))
        for i in range(len(vertices)):
            if i in isolated_nodes_set:
                num_isolated_nodes_passed += 1
            else:
                count_to_decrement[i] = num_isolated_nodes_passed

        for i, triangle in enumerate(triangles):
            start = time.time()
            node1, node2, node3 = triangle
            triangles[i][0] -= count_to_decrement[node1]
            triangles[i][1] -= count_to_decrement[node2]
            triangles[i][2] -= count_to_decrement[node3]
        for i, isolated_node in enumerate(isolated_nodes):
            vertex_list.pop(isolated_node - i)

    mesh_copy['vertices'] = np.array(vertex_list)

    return mesh_copy


def isolate_soma(key):

    full_start = time.time()

    print(str(key['segment_id']) +  ":")
    start = time.time()

    #find out if in Orphan Table or Regular Neuron Table
    if len(ta3p100.CoarseLabelFinal() & key) > 0:
        mesh = (ta3p100.CleansedMesh & key).fetch1()
        mesh_labels = (ta3p100.CoarseLabelFinal & key).fetch1()
    elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
        mesh = (ta3p100.CleansedMeshOrphan & key).fetch1()
        mesh_labels = (ta3p100.CoarseLabelOrphan & key).fetch1()
    else:
        raise Exception('neuron' + str(key["segment_id"]) + 
                        'not present in any labels!')
    
    print(key['segment_id'], "mesh fetched.", time.time() - start)
    start = time.time()
    
    #print(mesh['triangles'])
    myCounter = Counter(mesh_labels['triangles'])#.tolist())
    print(myCounter)
    
    keys = list(myCounter.keys())
    #print(len(keys))
    
    if len(keys) < 2 and keys[0] == 10:
        print("only error segments")
        key['n_vertices'] = 0
        key['n_triangles'] = 0
        key['vertices'] = np.array([])
        key['triangles'] = np.array([])
        
        print("This took ", time.time() - full_start, "seconds.")
        print()
        return key
    
    neighborhood = generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
    print(key['segment_id'] , "neighborhood generated.", time.time() - start)
    start = time.time()
    
    mesh = remove_non_soma(mesh,key,mesh_labels)
    print(key['segment_id'], "floating artifacts removed.", time.time() - start)
    start = time.time()

    mesh = remove_isolated_vertices(mesh)
    print(key['segment_id'], "isolated nodes removed.", time.time() - start)
    start = time.time()

    key['n_vertices'] = len(mesh['vertices'])
    key['n_triangles'] = len(mesh['triangles'])
    key['vertices'] = mesh['vertices']
    key['triangles'] = mesh['triangles']

    #self.insert1(key, skip_duplicates=True)
    print(key['segment_id'], "key successfully filtered.", time.time() - start)
    start = time.time()

    print("This took ", time.time() - full_start, "seconds.")
    print()
    return key

In [4]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_neurons"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [9]:
neuron_ID = 648518346349481014 #clean soma
neuron_ID = 648518346349481574 #not clean soma
key = dict(segmentation=2,segment_id=neuron_ID)

global_time = time.time()
#get the mesh with the error segments filtered away
start_time = time.time()
new_key = isolate_soma(key)
print(f"Step 1: Retrieving Mesh and removing error segments: {time.time() - start_time}")


648518346349481574:
648518346349481574 mesh fetched. 0.2089850902557373
Counter({3: 629259, 5: 167642, 6: 31973, 8: 2194})
648518346349481574 neighborhood generated. 11.907254695892334
648518346349481574 floating artifacts removed. 2.4120683670043945
648518346349481574 isolated nodes removed. 133.14017724990845
648518346349481574 key successfully filtered. 1.430511474609375e-05
This took  147.67125344276428 seconds.

Step 1: Retrieving Mesh and removing error segments: 148.0990767478943


In [10]:
new_key

{'segmentation': 2,
 'segment_id': 648518346349481574,
 'n_vertices': 75927,
 'n_triangles': 167642,
 'vertices': array([[437136.78125   , 205403.5625    ,  36192.5078125 ],
        [437164.40625   , 205340.15625   ,  36129.26171875],
        [437168.        , 205344.        ,  36320.        ],
        ...,
        [454705.53125   , 204518.453125  ,  38980.03125   ],
        [454695.65625   , 206035.8125    ,  38998.17578125],
        [454700.5625    , 204360.75      ,  38645.58984375]]),
 'triangles': array([[12393, 12253, 12355],
        [12393, 12355, 12442],
        [12393, 12442, 12623],
        ...,
        [41115, 41784, 41789],
        [20023, 18753, 18756],
        [18764, 20023, 18756]], dtype=uint32)}

In [ ]:
#run pymeshfix on mesh

In [11]:
 #print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]) + "ugly_soma",
                                                                  new_key["vertices"],
                                                                  new_key["triangles"])
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")




Done writing OFF file
Step 3: Writing shrinkwrap off file: 1.402228593826294


In [13]:
import pymeshfix
start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight
meshfix = pymeshfix.MeshFix(new_key["vertices"],new_key["triangles"])
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=True)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]) + "_pymeshfix",meshfix.v,meshfix.f)
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")

Step 2: Pymesh shrinkwrapping: 19.663155555725098
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.5451483726501465


In [23]:
n_vert_soma = (ta3p100.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'" & "component_index>0" )#.fetch("n_vertex_indices")

In [32]:
soma_extra = ta3p100.CompartmentFinal.ComponentFinal & (dj.U('segmentation', 'segment_id',"compartment_type") & n_vert_soma & "n_vertex_indices > 1000" )
soma_extra

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,n_vertex_indices,n_triangle_indices,vertex_indices preserved indices of each vertex of this sub-compartment,triangle_indices preserved indices of each triangle of this sub-compartment
2,648518346349475523,0.35,Soma,0,56648,126333,=BLOB=,=BLOB=
2,648518346349475523,0.35,Soma,1,13932,27717,=BLOB=,=BLOB=
2,648518346349475523,0.35,Soma,2,3,1,=BLOB=,=BLOB=
2,648518346349496278,0.35,Soma,0,3,1,=BLOB=,=BLOB=
2,648518346349496278,0.35,Soma,1,20202,40018,=BLOB=,=BLOB=


In [ ]:
n_vert_soma = (ta3p100.CompartmentFinalOrphan.ComponentFinalOrphan() & "compartment_type='Soma'" & "component_index>0")#.fetch("n_vertex_indices")

In [33]:
segment_id = 648518346349475523
segmentation = 2
query_key=dict(segment_id=segment_id,segmentation=segmentation)

vertices,triangles = (soma_extra & query_key).fetch("vertex_indices","triangle_indices")

In [34]:
triangles

array([array([ 85553,  85554,  85555, ..., 451246, 451247, 451248]),
       array([269373, 269374, 269375, ..., 409725, 409726, 409727]),
       array([337960])], dtype=object)

In [35]:
vertices

array([array([ 7555,  7563,  7636, ..., 91929, 91950, 91967], dtype=uint32),
       array([32768, 65538, 32770, ..., 32765, 32766, 32767], dtype=uint32),
       array([11433, 11691, 11956], dtype=uint32)], dtype=object)